In [1]:
import pandas as pd
import pyodbc
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,f1_score, jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler



import scipy.cluster.hierarchy as sch
from scipy.sparse import csr_matrix, vstack
import implicit
scaler = preprocessing.RobustScaler()

In [2]:
# Configuracion para la conexion de SQL
server = 'DITREXPRD'
database = 'DISAN_ASD_PRD'
username = 'analista1'
password = 'D1S4Nsoi7%'
# Crear la cadena de conexión
cadena_conexion = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password

In [4]:
# Conectarse a BD
conexion = pyodbc.connect(cadena_conexion)

# Ruta y lectura del archivo .sql de Interaccion clientes producto actual
sql_file_path_int = 'Clientes_Actuales_Compras.sql'
with open(sql_file_path_int, 'r') as file:
    sql_query_int = file.read()

# Ruta y lectura del archivo .sql de carateristicas de clientes  actual
sql_file_path_Cli_car_act = 'Caracteristicas_Clientes_Actuales.sql'
with open(sql_file_path_Cli_car_act, 'r') as file:
    sql_query_Cli_car_act = file.read()

# Ruta y lectura del archivo .sql de  carateristicas de clientes  nuevos
sql_file_path_Cli_car_new = 'Caracteristicas_Clientes_Nuevos.sql'
with open(sql_file_path_Cli_car_new, 'r') as file:
    sql_file_path_Cli_car_new = file.read()
    
# Ruta y lectura del archivo .sql de  carateristicas de productos
sql_file_path_prod_car = 'Caracteristicas_Productos.sql'
with open(sql_file_path_prod_car, 'r') as file:
    sql_file_path_prod_car = file.read()

# Ruta y lectura del archivo .sql de Interaccion clientes producto actual
sql_file_path_compra_new = 'Clientes_Nuevos_Compras.sql'
with open(sql_file_path_compra_new, 'r') as file:
    sql_query_compra_new = file.read()
    

# Ejecutar la consulta y cargar el resultado en un DataFrame de pandas
df_inter_cliact_prod = pd.read_sql_query(sql_query_int, conexion)
df_caract_clie_act = pd.read_sql_query(sql_query_Cli_car_act, conexion)
df_caract_clie_new = pd.read_sql_query(sql_file_path_Cli_car_new, conexion)
df_caract_prod = pd.read_sql_query(sql_file_path_prod_car, conexion)
df_inter_clinew_prod = pd.read_sql_query(sql_query_compra_new, conexion)

C:\Users\pablo.isaza\AppData\Local\Temp\ipykernel_9656\1382735531.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_inter_cliact_prod = pd.read_sql_query(sql_query_int, conexion)
C:\Users\pablo.isaza\AppData\Local\Temp\ipykernel_9656\1382735531.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_caract_clie_act = pd.read_sql_query(sql_query_Cli_car_act, conexion)
C:\Users\pablo.isaza\AppData\Local\Temp\ipykernel_9656\1382735531.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_caract_clie_new = pd.read_sql_q

In [5]:
df_inter_cliact_prod

,cllienteid,materialid,JerMatDescripcion1,JerMatDescripcion3,Portafolio
0,0,64,CELULOSICOS - PINTURAS,CELLOSIZE QP,1
1,129,72,CELULOSICOS - PINTURAS,CELLOSIZE QP,1
2,406,72,CELULOSICOS - PINTURAS,CELLOSIZE QP,1
3,406,1473,TENSOACTIVOS ESP. - D,ECOSURF,1
4,406,19440,POLIMERO ACRILICO,SUPRACARE,1
...,...,...,...,...,...
81881,10514,8061,ACEITE MINERAL,ACEITE MINERAL LIVIANO,0
81882,528,1387,ALCANOLAMINAS,TEA,0
81883,5796,2414,LATEX DE CAUCHO NATURAL,LATEX DE CAUCHO NATURAL,0
81884,18194,243,PARAFINA CHINA,PARAFINA CHINA SR,0


In [6]:
df_inter_clinew_prod

,cllienteid,materialid,JerMatDescripcion1,JerMatDescripcion3
0,23934,23178,ACIDO SULFONICO,ACIDO SULFONICO LINEAL
1,24411,23178,ACIDO SULFONICO,ACIDO SULFONICO LINEAL
2,28887,23178,ACIDO SULFONICO,ACIDO SULFONICO LINEAL
3,23934,23178,ACIDO SULFONICO,ACIDO SULFONICO LINEAL
4,24411,23179,ACIDO SULFONICO,ACIDO SULFONICO LINEAL
...,...,...,...,...
2403,23787,22622,LINEA EVONIK,ACTIVOS NATURALES
2404,23787,22622,LINEA EVONIK,ACTIVOS NATURALES
2405,23786,22881,LINEA EVONIK,EXTRACTOS NATURALES
2406,23786,22899,LINEA EVONIK,EXTRACTOS NATURALES


In [7]:
df_caract_clie_act

,clienteid,industria,subindustria,segmento,clasificacion,tipo_segmento
0,3,1,5,9,01,D
1,4,3,11,9,05,D
2,5,9,27,9,04,D
3,6,9,27,9,05,D
4,19,7,16,9,05,D
...,...,...,...,...,...,...
15157,48528,10,11,9,03,D
15158,48529,10,29,9,02,D
15159,48544,4,12,8,02,C
15160,48545,15,51,8,,C


In [8]:
df_caract_clie_new

,clienteid,industria,subindustria,segmento,clasificacion,tipo_segmento
0,23617,3,11,9,03,D
1,23705,11,30,9,05,D
2,23712,4,12,6,05,A
3,23718,7,16,9,05,D
4,23719,4,12,9,05,D
...,...,...,...,...,...,...
178,48024,5,13,8,02,C
179,48025,5,13,8,02,C
180,48038,5,13,11,02,
181,48365,4,12,8,02,C


In [9]:
df_caract_prod

,materialid,TipoMaterialId,GrupoArticuloId,JerMatDescripcion1,JerMatDescripcion3,Portafolo
0,56,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1
1,61,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1
2,63,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1
3,64,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1
4,65,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1
...,...,...,...,...,...,...
14090,23701,65,95,FRAGANCIAS Y AROMAS,FRUTAL AROMÁTICA,0
14091,23710,40,3,MUESTRA,MUESTRA,0
14092,23711,40,3,MUESTRA,MUESTRA,0
14093,23712,40,3,MUESTRA,MUESTRA,0


In [10]:
df_productos = df_caract_prod
# Tenemos que crear los id de la tabla de productos, que van hacer los distinitos de jerarquiaproducto 1 y 3
df_productos = df_productos.drop(columns=['materialid'])
df_productos.sort_values(by='Portafolo', ascending=False)
df_productos = df_productos.drop_duplicates(subset=['TipoMaterialId', 'GrupoArticuloId','JerMatDescripcion1', 'JerMatDescripcion3']).reset_index()
df_productos = df_productos.drop(columns=['index'])
df_productos['prod_id'] = df_productos.index
df_productos


,TipoMaterialId,GrupoArticuloId,JerMatDescripcion1,JerMatDescripcion3,Portafolo,prod_id
0,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1,0
1,14,20,CELULOSICOS - PINTURAS,CELLOSIZE WP,1,1
2,14,80,TENSOACTIVOS ESP. - D,ECOSURF,1,2
3,14,20,CELULOSICOS - PINTURAS,CELLOSIZE ER,1,3
4,40,20,MUESTRA,MUESTRA,1,4
...,...,...,...,...,...,...
1404,70,92,SOLUBLES,ZINC,0,1404
1405,14,24,ACTIVOS FARMA APIS,OFTALMICOS,0,1405
1406,65,95,FRAGANCIAS Y AROMAS,FRUTAL,0,1406
1407,73,93,FOSFATOS,PIROFOSFATO ACIDO DE SODIO,0,1407


In [11]:
# Tenemos que unir los df caracteristica, con el id de prod_id para homologar esos materiales
df_caract_prod_2 = pd.merge(df_caract_prod, df_productos[['JerMatDescripcion1', 'JerMatDescripcion3' ,'prod_id']], on=['JerMatDescripcion1', 'JerMatDescripcion3'], how= 'left')
df_caract_prod_2

,materialid,TipoMaterialId,GrupoArticuloId,JerMatDescripcion1,JerMatDescripcion3,Portafolo,prod_id
0,56,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1,0
1,61,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1,0
2,63,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1,0
3,64,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1,0
4,65,14,20,CELULOSICOS - PINTURAS,CELLOSIZE QP,1,0
...,...,...,...,...,...,...,...
443298,23713,40,3,MUESTRA,MUESTRA,0,889
443299,23713,40,3,MUESTRA,MUESTRA,0,895
443300,23713,40,3,MUESTRA,MUESTRA,0,897
443301,23713,40,3,MUESTRA,MUESTRA,0,919


In [12]:
df_compra = df_inter_cliact_prod
df_compra = df_compra[df_compra['Portafolio']==1]
df_compra = df_compra.drop(columns=['JerMatDescripcion1','JerMatDescripcion3', 'Portafolio'])
df_compra = pd.merge(df_compra,df_caract_prod_2[['materialid', 'prod_id']], on = 'materialid', how = 'left' )
df_compra = df_compra.dropna()
df_compra = df_compra.drop(columns='materialid').reset_index()
df_compra =  df_compra.drop(columns='index')
df_compra = df_compra.drop_duplicates()
df_compra

,cllienteid,prod_id
0,0,0.0
1,129,0.0
3,406,0.0
5,406,2.0
7,406,16.0
...,...,...
1701,47073,0.0
1705,47403,0.0
1706,47429,0.0
1707,47861,9.0


In [13]:
df_compra_nueva = df_inter_clinew_prod
df_compra_nueva = pd.merge(df_compra_nueva,df_caract_prod_2[['materialid', 'prod_id']], on = 'materialid', how = 'left' )
df_compra_nueva = df_compra_nueva.drop(columns=['JerMatDescripcion1','JerMatDescripcion3'])
df_compra_nueva = df_compra_nueva.dropna()
df_compra_nueva = df_compra_nueva.drop(columns='materialid').reset_index()
df_compra_nueva =  df_compra_nueva.drop(columns='index')
df_compra_nueva = df_compra_nueva.drop_duplicates()
df_compra_nueva


,cllienteid,prod_id
0,23934,135.0
1,23934,434.0
2,23934,585.0
3,23934,1293.0
4,24411,135.0
...,...,...
5124,23792,1204.0
5125,23786,1192.0
5126,25490,1204.0
5130,23769,1251.0


In [14]:
# Función para renombrar las columnas en el one-hot encoding
def custom_one_hot(df, column_name, unique_ids):
    one_hot_df = pd.get_dummies(df[column_name])
    one_hot_df = one_hot_df.reindex(columns=unique_ids, fill_value=0)
    # Renombrar las columnas agregando el nombre de la columna original
    one_hot_df.columns = [f'{column_name}-{col}' for col in one_hot_df.columns]
    return one_hot_df


In [15]:
# Tenemos que tener las mismas cracarteristicas para un modelo de recomendacion, por lo que tenemos que tener en la

lista_ids_unicos_ind = pd.concat([df_caract_clie_act['industria'],df_caract_clie_new['industria']]).unique()
lista_ids_unicos_subind = pd.concat([df_caract_clie_act['subindustria'],df_caract_clie_new['subindustria']]).unique()
lista_ids_unicos_seg =  pd.concat([df_caract_clie_act['segmento'],df_caract_clie_new['segmento']]).unique()
lista_ids_unicos_cla = pd.concat([df_caract_clie_act['clasificacion'],df_caract_clie_new['clasificacion']]).unique()
lista_ids_unicos_tpseg =  pd.concat([df_caract_clie_act['tipo_segmento'],df_caract_clie_new['tipo_segmento']]).unique()

#Re dimensionar Region Actual
one_hot_region1_ind = custom_one_hot(df_caract_clie_act, 'industria', lista_ids_unicos_ind)
one_hot_region1_subind  = custom_one_hot(df_caract_clie_act, 'subindustria', lista_ids_unicos_subind)
one_hot_region1_seg = custom_one_hot(df_caract_clie_act, 'segmento', lista_ids_unicos_seg)
one_hot_region1_cla = custom_one_hot(df_caract_clie_act, 'clasificacion', lista_ids_unicos_cla)
one_hot_region1_tpseg = custom_one_hot(df_caract_clie_act, 'tipo_segmento', lista_ids_unicos_tpseg)

df_cie_encod_act = pd.concat([one_hot_region1_ind, one_hot_region1_subind, one_hot_region1_seg, one_hot_region1_cla,one_hot_region1_tpseg], axis=1)

df_cie_encod_act

,industria-1,industria-3,industria-9,industria-7,industria-10,industria-16,industria-5,industria-14,industria-4,industria-2,...,clasificacion-06,clasificacion-1,clasificacion-,tipo_segmento-D,tipo_segmento-C,tipo_segmento-,tipo_segmento-A,tipo_segmento-E,tipo_segmento-B,tipo_segmento-3
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15157,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
15158,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
15159,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False
15160,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,False


In [16]:
#Re dimensionar Region Actual
one_hot_region2_ind = custom_one_hot(df_caract_clie_new, 'industria', lista_ids_unicos_ind)
one_hot_region2_subind  = custom_one_hot(df_caract_clie_new, 'subindustria', lista_ids_unicos_subind)
one_hot_region2_seg = custom_one_hot(df_caract_clie_new, 'segmento', lista_ids_unicos_seg)
one_hot_region2_cla = custom_one_hot(df_caract_clie_new, 'clasificacion', lista_ids_unicos_cla)
one_hot_region2_tpseg = custom_one_hot(df_caract_clie_new, 'tipo_segmento', lista_ids_unicos_tpseg)


df_cie_encod_new = pd.concat([one_hot_region2_ind, one_hot_region2_subind, one_hot_region2_seg, one_hot_region2_cla,one_hot_region2_tpseg], axis=1)
df_cie_encod_new


,industria-1,industria-3,industria-9,industria-7,industria-10,industria-16,industria-5,industria-14,industria-4,industria-2,...,clasificacion-06,clasificacion-1,clasificacion-,tipo_segmento-D,tipo_segmento-C,tipo_segmento-,tipo_segmento-A,tipo_segmento-E,tipo_segmento-B,tipo_segmento-3
0,0,True,0,False,0,0,False,False,False,0,...,0,0,0,True,False,False,False,False,False,0
1,0,False,0,False,0,0,False,False,False,0,...,0,0,0,True,False,False,False,False,False,0
2,0,False,0,False,0,0,False,False,True,0,...,0,0,0,False,False,False,True,False,False,0
3,0,False,0,True,0,0,False,False,False,0,...,0,0,0,True,False,False,False,False,False,0
4,0,False,0,False,0,0,False,False,True,0,...,0,0,0,True,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0,False,0,False,0,0,True,False,False,0,...,0,0,0,False,True,False,False,False,False,0
179,0,False,0,False,0,0,True,False,False,0,...,0,0,0,False,True,False,False,False,False,0
180,0,False,0,False,0,0,True,False,False,0,...,0,0,0,False,False,True,False,False,False,0
181,0,False,0,False,0,0,False,False,True,0,...,0,0,0,False,True,False,False,False,False,0


In [17]:
# Como vemos, ya las dos matrices tienen la misma dimension en columnas, ahora es el turno para las caracteristicas de los productos de la region A
df_productos_encoded = df_productos
df_productos_encoded[['TipoMaterialId', 'GrupoArticuloId', 'Portafolo']] = df_productos_encoded[['TipoMaterialId', 'GrupoArticuloId', 'Portafolo']].astype(str)
df_productos_encoded = pd.get_dummies(df_productos_encoded)
df_productos_encoded


,prod_id,TipoMaterialId_14,TipoMaterialId_40,TipoMaterialId_64,TipoMaterialId_65,TipoMaterialId_66,TipoMaterialId_70,TipoMaterialId_73,GrupoArticuloId_1,GrupoArticuloId_10,...,JerMatDescripcion3_YOGUEST,JerMatDescripcion3_ZEA GUM,JerMatDescripcion3_ZINC,JerMatDescripcion3_ÁCIDO ALQUIBENCEN SULFÓNICO LINEAL,JerMatDescripcion3_ÁCIDO ETILENEDIAMINETRAACÉTICO TETRASODI,JerMatDescripcion3_ÁCIDO GLUCÓNICO,JerMatDescripcion3_ÁMBAR FLORAL,JerMatDescripcion3_ÁMBAR FOUGÉRE,Portafolo_0,Portafolo_1
0,0,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,2,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,3,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,4,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,1404,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
1405,1405,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1406,1406,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1407,1407,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False


In [18]:
# Ya tenemos La matrices de carcterisiticas de los productos y de los clientes, ahora vamos a empezar comparando que clientes son parecidos en carcteristicas
# y para esto utilizaremos la similitud de coseno

usuario_sim = cosine_similarity( df_cie_encod_act,df_cie_encod_new)
print(usuario_sim.shape)

(15162, 183)


In [19]:
prod_sim = cosine_similarity(df_productos_encoded)
prod_sim.shape

(1409, 1409)

In [20]:
matriz_usu_prod_act = df_compra.pivot_table(index='cllienteid', columns='prod_id',  aggfunc='size', fill_value=0)
matriz_usu_prod_new = df_compra_nueva.pivot_table(index='cllienteid', columns='prod_id',  aggfunc='size', fill_value=0)
print(matriz_usu_prod_act.shape)
print(matriz_usu_prod_new.shape)

(727, 15)
(110, 91)


In [21]:
def recommend_products_region2(matriz_usu_prod_act, matriz_usu_prod_new, usuario_sim, prod_sim):
    num_customers_r1, num_products_r1 = matriz_usu_prod_act.shape
    num_customers_r2, num_products_r2 = matriz_usu_prod_new.shape

    # Matriz para almacenar las puntuaciones de recomendación para cada cliente de la región 2
    recommendation_scores = np.zeros((num_customers_r2, num_products_r1))

    # Para cada cliente en la región 2
    for customer_r2_idx in range(num_customers_r2):
        # Para cada producto de la región 1
        for product_r1_idx in range(num_products_r1):
            score = 0
            # Combinar la similitud de los clientes y la similitud de los productos
            for customer_r1_idx in range(num_customers_r1):
                # Ponderar la similitud del cliente de la región 2 con el cliente de la región 1
                customer_similarity = usuario_sim[customer_r1_idx, customer_r2_idx]  # Acceso directo en numpy
                #print(customer_similarity)
                #print(customer_r1_idx, product_r1_idx)
                # Ponderar la similitud del producto de la región 1 con los productos que el cliente de la región 1 ha comprado
                for product_r2_idx in range(num_products_r2):
                    # Acceso directo a la matriz de interacción
                    if matriz_usu_prod_act.iloc[customer_r1_idx, product_r1_idx] == 1: # Si el cliente de la región 1 compró el producto r1
                        #print(product_r1_idx,product_r2_idx)
                        product_similarity = prod_sim[product_r1_idx, product_r2_idx]  # Acceso directo en numpy
                        score += customer_similarity * product_similarity
            
            # Almacenar la puntuación de recomendación para ese cliente y producto
            recommendation_scores[customer_r2_idx, product_r1_idx] = score
    
    return recommendation_scores




In [22]:
# Supongamos que estas son tus matrices originales de interacción
# Convertir las matrices a formato disperso (sparse matrices)
matriz_usu_prod_act_sparse = csr_matrix(matriz_usu_prod_act)
matriz_usu_prod_new_sparse = csr_matrix(matriz_usu_prod_new)

In [23]:
def recommend_products_region2_sparse(matriz_usu_prod_act_sparse, matriz_usu_prod_new_sparse, usuario_sim, prod_sim):
    num_customers_r1, num_products_r1 = matriz_usu_prod_act_sparse.shape
    num_customers_r2, num_products_r2 = matriz_usu_prod_new_sparse.shape

    # Matriz dispersa para almacenar las puntuaciones de recomendación para cada cliente de la región 2
    recommendation_scores_sparse = csr_matrix((num_customers_r2, num_products_r1))

    # Para cada cliente en la región 2
    for customer_r2_idx in range(num_customers_r2):
        # Para cada producto de la región 1
        print(customer_r2_idx)
        for product_r1_idx in range(num_products_r1):
            score = 0
            # Combinar la similitud de los clientes y la similitud de los productos
            for customer_r1_idx in range(num_customers_r1):
                # Ponderar la similitud del cliente de la región 2 con el cliente de la región 1
                customer_similarity = usuario_sim[customer_r1_idx, customer_r2_idx]  # Acceso directo en numpy
                #print(customer_similarity)
                #print(customer_r1_idx, product_r1_idx)
                # Ponderar la similitud del producto de la región 1 con los productos que el cliente de la región 1 ha comprado
                for product_r2_idx in range(num_products_r2):
                    # Acceso directo a la matriz de interacción
                    if matriz_usu_prod_act.iloc[customer_r1_idx, product_r1_idx] == 1: # Si el cliente de la región 1 compró el producto r1
                        #print(product_r1_idx,product_r2_idx)
                        product_similarity = prod_sim[product_r1_idx, product_r2_idx]  # Acceso directo en numpy
                        score += customer_similarity * product_similarity
            
            # Almacenar la puntuación de recomendación para ese cliente y producto
            recommendation_scores_sparse[customer_r2_idx, product_r1_idx] = score
    
    return recommendation_scores_sparse


In [24]:
matriz_usu_prod_act.iloc[0,0]

1

In [25]:
# Calcular las puntuaciones de recomendación para clientes de la región 2
recommendation_scores_r2 = recommend_products_region2_sparse(matriz_usu_prod_act, matriz_usu_prod_new, usuario_sim, prod_sim)

0


C:\Users\pablo.isaza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\sparse\_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109


In [32]:
print(recommendation_scores_r2.shape)


(110, 15)


In [27]:
df_recommendation_scores = pd.DataFrame(recommendation_scores_r2)

print(df_recommendation_scores)

                                                     0
0      (0, 0)\t1250.3566484982919\n  (0, 1)\t47.992...
1      (0, 0)\t1231.174049985763\n  (0, 1)\t47.9920...
2      (0, 0)\t367.08518062604696\n  (0, 1)\t15.997...
3      (0, 0)\t1247.740839610223\n  (0, 1)\t47.9920...
4      (0, 0)\t1291.3376544114278\n  (0, 1)\t47.992...
..                                                 ...
105    (0, 0)\t371.44486210617816\n  (0, 1)\t0.0\n ...
106    (0, 0)\t383.65197025050935\n  (0, 1)\t0.0\n ...
107    (0, 0)\t383.65197025050935\n  (0, 1)\t0.0\n ...
108    (0, 0)\t1171.8823818561168\n  (0, 1)\t31.994...
109    (0, 0)\t383.65197025050935\n  (0, 1)\t0.0\n ...

[110 rows x 1 columns]


In [28]:
# Si recommendation_scores_r2 es una matriz dispersa
recommendation_scores_r2_dense = recommendation_scores_r2.todense()

# Crear el DataFrame
df_recommendation_scores = pd.DataFrame(recommendation_scores_r2_dense, 
                                        columns=[f'Producto_r1_{i+1}' for i in range(recommendation_scores_r2_dense.shape[1])], 
                                        index=[f'Cliente_r2_{i+1}' for i in range(recommendation_scores_r2_dense.shape[0])])

print("Puntuaciones de recomendación de productos de la región Actual para clientes de la región Nueva:")
print(df_recommendation_scores)

Puntuaciones de recomendación de productos de la región Actual para clientes de la región Nueva:
                Producto_r1_1  Producto_r1_2  Producto_r1_3  Producto_r1_4  \
Cliente_r2_1      1250.356648      47.992096     971.847442     659.860527   
Cliente_r2_2      1231.174050      47.992096    1230.186636     513.224854   
Cliente_r2_3       367.085181      15.997365     393.659723     190.626375   
Cliente_r2_4      1247.740840      47.992096    1254.790368     615.869825   
Cliente_r2_5      1291.337654      47.992096    1279.394101     630.533393   
...                       ...            ...            ...            ...   
Cliente_r2_106     371.444862       0.000000     123.018664     366.589182   
Cliente_r2_107     383.651970       0.000000     135.320530     337.262047   
Cliente_r2_108     383.651970       0.000000     135.320530     337.262047   
Cliente_r2_109    1171.882382      31.994731    1057.960507     542.551989   
Cliente_r2_110     383.651970       0.000000 

In [29]:
df_recommendation_scores

,Producto_r1_1,Producto_r1_2,Producto_r1_3,Producto_r1_4,Producto_r1_5,Producto_r1_6,Producto_r1_7,Producto_r1_8,Producto_r1_9,Producto_r1_10,Producto_r1_11,Producto_r1_12,Producto_r1_13,Producto_r1_14,Producto_r1_15
Cliente_r2_1,1250.356648,47.992096,971.847442,659.860527,1460.912099,408.382958,785.834330,746.789177,359.863936,242.345008,34.776859,34.891393,314.796613,770.952310,772.081688
Cliente_r2_2,1231.174050,47.992096,1230.186636,513.224854,1366.659705,457.388913,769.114450,695.871733,394.136691,190.413935,52.165288,34.891393,332.285313,718.387380,719.439755
Cliente_r2_3,367.085181,15.997365,393.659723,190.626375,424.135771,130.682547,167.198794,220.642257,154.227401,86.551788,52.165288,0.000000,157.398306,227.781364,228.115044
Cliente_r2_4,1247.740840,47.992096,1254.790368,615.869825,1476.620831,490.059550,852.713847,661.926770,428.409447,190.413935,52.165288,34.891393,297.307912,683.344093,684.345133
Cliente_r2_5,1291.337654,47.992096,1279.394101,630.533393,1492.329563,490.059550,769.114450,695.871733,462.682203,225.034650,86.942147,34.891393,367.262715,718.387380,719.439755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cliente_r2_106,371.444862,0.000000,123.018664,366.589182,329.883377,114.347228,83.599397,339.449626,85.681889,34.620715,34.776859,0.000000,174.887007,350.432868,350.946222
Cliente_r2_107,383.651970,0.000000,135.320530,337.262047,282.757180,114.347228,100.319276,305.504663,34.272756,0.000000,0.000000,0.000000,139.909606,315.389581,315.851600
Cliente_r2_108,383.651970,0.000000,135.320530,337.262047,282.757180,114.347228,100.319276,305.504663,34.272756,0.000000,0.000000,0.000000,139.909606,315.389581,315.851600
Cliente_r2_109,1171.882382,31.994731,1057.960507,542.551989,1288.116044,424.718277,702.234933,577.064364,377.000313,155.793219,34.776859,34.891393,244.841810,595.735876,596.608577


In [30]:
# Para obtener los nombres de los productos (columnas) con los 5 valores más altos por fila (cliente)
top_5_columnas_por_fila = df_recommendation_scores.apply(lambda x: x.nlargest(5).index.tolist(), axis=1).tolist()

# Mostramos el resultado
for i, top in enumerate(top_5_columnas_por_fila, 1):
    print(f"Cliente_r2_{i}: {top}")


Cliente_r2_1: ['Producto_r1_5', 'Producto_r1_1', 'Producto_r1_3', 'Producto_r1_7', 'Producto_r1_15']
Cliente_r2_2: ['Producto_r1_5', 'Producto_r1_1', 'Producto_r1_3', 'Producto_r1_7', 'Producto_r1_15']
Cliente_r2_3: ['Producto_r1_5', 'Producto_r1_3', 'Producto_r1_1', 'Producto_r1_15', 'Producto_r1_14']
Cliente_r2_4: ['Producto_r1_5', 'Producto_r1_3', 'Producto_r1_1', 'Producto_r1_7', 'Producto_r1_15']
Cliente_r2_5: ['Producto_r1_5', 'Producto_r1_1', 'Producto_r1_3', 'Producto_r1_7', 'Producto_r1_15']
Cliente_r2_6: ['Producto_r1_1', 'Producto_r1_4', 'Producto_r1_15', 'Producto_r1_14', 'Producto_r1_8']
Cliente_r2_7: ['Producto_r1_5', 'Producto_r1_1', 'Producto_r1_3', 'Producto_r1_7', 'Producto_r1_15']
Cliente_r2_8: ['Producto_r1_5', 'Producto_r1_1', 'Producto_r1_3', 'Producto_r1_7', 'Producto_r1_4']
Cliente_r2_9: ['Producto_r1_1', 'Producto_r1_4', 'Producto_r1_15', 'Producto_r1_14', 'Producto_r1_8']
Cliente_r2_10: ['Producto_r1_5', 'Producto_r1_1', 'Producto_r1_3', 'Producto_r1_7', 'Prod

In [31]:
# Obtener los nombres de los clientes (filas) con los 5 valores más altos por cada columna (producto)
top_5_filas_por_columna = [df_recommendation_scores[col].nlargest(5).index.tolist() for col in df_recommendation_scores.columns]

# Mostrar el resultado
for col_name, top_rows in zip(df_recommendation_scores.columns, top_5_filas_por_columna):
    print(f"{col_name}: {top_rows}")

Producto_r1_1: ['Cliente_r2_51', 'Cliente_r2_5', 'Cliente_r2_29', 'Cliente_r2_10', 'Cliente_r2_47']
Producto_r1_2: ['Cliente_r2_2', 'Cliente_r2_4', 'Cliente_r2_5', 'Cliente_r2_10', 'Cliente_r2_29']
Producto_r1_3: ['Cliente_r2_51', 'Cliente_r2_5', 'Cliente_r2_29', 'Cliente_r2_10', 'Cliente_r2_4']
Producto_r1_4: ['Cliente_r2_11', 'Cliente_r2_18', 'Cliente_r2_20', 'Cliente_r2_22', 'Cliente_r2_25']
Producto_r1_5: ['Cliente_r2_5', 'Cliente_r2_29', 'Cliente_r2_10', 'Cliente_r2_4', 'Cliente_r2_1']
Producto_r1_6: ['Cliente_r2_4', 'Cliente_r2_5', 'Cliente_r2_29', 'Cliente_r2_51', 'Cliente_r2_10']
Producto_r1_7: ['Cliente_r2_4', 'Cliente_r2_1', 'Cliente_r2_51', 'Cliente_r2_10', 'Cliente_r2_2']
Producto_r1_8: ['Cliente_r2_1', 'Cliente_r2_2', 'Cliente_r2_5', 'Cliente_r2_29', 'Cliente_r2_10']
Producto_r1_9: ['Cliente_r2_5', 'Cliente_r2_29', 'Cliente_r2_7', 'Cliente_r2_19', 'Cliente_r2_35']
Producto_r1_10: ['Cliente_r2_1', 'Cliente_r2_5', 'Cliente_r2_29', 'Cliente_r2_84', 'Cliente_r2_101']
Producto_

### Validar el algoritmo

In [44]:
# primero se debe de calcular nuevamente la matriz de usuarios pero solo la de los clientes de la region 1

usuario_sim_r1 = cosine_similarity(df_cie_encod_act)

In [42]:
def recommend_products_region1_subset(matriz_usu_prod_act_sparse, usuario_sim, prod_sim, num_customers=15):
    num_customers_r1, num_products_r1 = matriz_usu_prod_act_sparse.shape
    
    # Seleccionar 15 clientes de la región 1 al azar
    selected_customers_r1 = np.random.choice(num_customers_r1, num_customers, replace=False)

    # Matriz dispersa para almacenar las puntuaciones de recomendación para los clientes seleccionados
    recommendation_scores_sparse = csr_matrix((num_customers, num_products_r1))

    # Para cada cliente seleccionado en la región 1
    for idx, customer_r1_idx in enumerate(selected_customers_r1):
        # Para cada producto de la región 1
        for product_r1_idx in range(num_products_r1):
            score = 0
            # Ponderar la similitud entre el cliente seleccionado y otros clientes de la región 1
            for other_customer_r1_idx in range(num_customers_r1):
                # Acceder a la similitud solo de los clientes seleccionados
                customer_similarity = usuario_sim[customer_r1_idx, other_customer_r1_idx]
                
                # Verificar si el otro cliente de la región 1 compró el producto r1
                if matriz_usu_prod_act_sparse[other_customer_r1_idx, product_r1_idx] == 1:  # Si lo compró
                    # Ponderar la similitud del producto
                    product_similarity = prod_sim[product_r1_idx, product_r1_idx]
                    score += customer_similarity * product_similarity
            
            # Almacenar la puntuación de recomendación para ese cliente seleccionado y producto
            recommendation_scores_sparse[idx, product_r1_idx] = score
    
    return recommendation_scores_sparse, selected_customers_r1





In [45]:
recommendation_scores_r1_validation, selected_customers_r1 = recommend_products_region1_subset(matriz_usu_prod_act_sparse, usuario_sim_r1, prod_sim)

C:\Users\pablo.isaza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\sparse\_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [47]:
# Si recommendation_scores_r2 es una matriz dispersa
recommendation_scores_r1_validation_dense = recommendation_scores_r1_validation.todense()

# Crear el DataFrame
df_recommendation_scores_val_r1 = pd.DataFrame(recommendation_scores_r1_validation_dense, 
                                        columns=[f'Producto_r1_{i+1}' for i in range(recommendation_scores_r1_validation_dense.shape[1])], 
                                        index=[f'Cliente_val_{i+1}' for i in range(recommendation_scores_r1_validation_dense.shape[0])])

print("Puntuaciones de recomendación de productos de la región Actual para clientes de la región Actual:")
print(df_recommendation_scores_val_r1)

Puntuaciones de recomendación de productos de la región Actual para clientes de la región Actual:
                Producto_r1_1  Producto_r1_2  Producto_r1_3  Producto_r1_4  \
Cliente_val_1           318.8            1.2           22.4            8.2   
Cliente_val_2           254.6            0.8           16.0            8.8   
Cliente_val_3           318.6            1.6           20.6            7.8   
Cliente_val_4           296.6            1.2           20.2            7.8   
Cliente_val_5           255.0            0.8           17.0            6.8   
Cliente_val_6           318.6            1.6           20.6            7.8   
Cliente_val_7           105.8            0.8            6.2            2.2   
Cliente_val_8           274.0            1.2           17.4            6.4   
Cliente_val_9           297.8            1.2           21.2            7.0   
Cliente_val_10          257.8            1.2           15.4            7.0   
Cliente_val_11          285.4            1.2

In [48]:
df_recommendation_scores_val_r1

,Producto_r1_1,Producto_r1_2,Producto_r1_3,Producto_r1_4,Producto_r1_5,Producto_r1_6,Producto_r1_7,Producto_r1_8,Producto_r1_9,Producto_r1_10,Producto_r1_11,Producto_r1_12,Producto_r1_13,Producto_r1_14,Producto_r1_15
Cliente_val_1,318.8,1.2,22.4,8.2,18.8,6.4,9.6,7.6,5.0,2.2,0.6,0.4,3.6,7.6,7.6
Cliente_val_2,254.6,0.8,16.0,8.8,16.4,5.0,7.8,6.8,3.8,1.6,0.4,0.4,2.8,6.8,6.8
Cliente_val_3,318.6,1.6,20.6,7.8,21.0,5.6,10.8,9.4,5.0,3.4,0.6,0.4,4.2,9.4,9.4
Cliente_val_4,296.6,1.2,20.2,7.8,18.6,5.6,9.6,7.8,4.8,2.2,0.6,0.4,3.8,7.8,7.8
Cliente_val_5,255.0,0.8,17.0,6.8,16.6,4.8,8.2,6.6,4.2,1.8,0.4,0.4,2.6,6.6,6.6
Cliente_val_6,318.6,1.6,20.6,7.8,21.0,5.6,10.8,9.4,5.0,3.4,0.6,0.4,4.2,9.4,9.4
Cliente_val_7,105.8,0.8,6.2,2.2,7.4,1.2,3.6,4.2,1.4,2.2,0.2,0.0,2.2,4.2,4.2
Cliente_val_8,274.0,1.2,17.4,6.4,15.8,5.6,9.4,7.4,5.0,1.8,0.4,0.4,2.6,7.4,7.4
Cliente_val_9,297.8,1.2,21.2,7.0,18.6,6.2,9.6,8.2,4.8,2.2,0.6,0.4,3.4,8.2,8.2
Cliente_val_10,257.8,1.2,15.4,7.0,17.4,4.4,9.0,7.6,4.0,2.8,0.4,0.4,3.2,7.6,7.6


In [49]:
# Para obtener los nombres de los productos (columnas) con los 5 valores más altos por fila (cliente)
top_5_columnas_por_fila_r1 = df_recommendation_scores_val_r1.apply(lambda x: x.nlargest(5).index.tolist(), axis=1).tolist()

# Mostramos el resultado
for i, top in enumerate(top_5_columnas_por_fila_r1, 1):
    print(f"Cliente_val_{i}: {top}")


Cliente_val_1: ['Producto_r1_1', 'Producto_r1_3', 'Producto_r1_5', 'Producto_r1_7', 'Producto_r1_4']
Cliente_val_2: ['Producto_r1_1', 'Producto_r1_5', 'Producto_r1_3', 'Producto_r1_4', 'Producto_r1_7']
Cliente_val_3: ['Producto_r1_1', 'Producto_r1_5', 'Producto_r1_3', 'Producto_r1_7', 'Producto_r1_14']
Cliente_val_4: ['Producto_r1_1', 'Producto_r1_3', 'Producto_r1_5', 'Producto_r1_7', 'Producto_r1_14']
Cliente_val_5: ['Producto_r1_1', 'Producto_r1_3', 'Producto_r1_5', 'Producto_r1_7', 'Producto_r1_4']
Cliente_val_6: ['Producto_r1_1', 'Producto_r1_5', 'Producto_r1_3', 'Producto_r1_7', 'Producto_r1_14']
Cliente_val_7: ['Producto_r1_1', 'Producto_r1_5', 'Producto_r1_3', 'Producto_r1_14', 'Producto_r1_8']
Cliente_val_8: ['Producto_r1_1', 'Producto_r1_3', 'Producto_r1_5', 'Producto_r1_7', 'Producto_r1_14']
Cliente_val_9: ['Producto_r1_1', 'Producto_r1_3', 'Producto_r1_5', 'Producto_r1_7', 'Producto_r1_8']
Cliente_val_10: ['Producto_r1_1', 'Producto_r1_5', 'Producto_r1_3', 'Producto_r1_7', '

In [50]:
# Obtener los nombres de los clientes (filas) con los 5 valores más altos por cada columna (producto)
top_5_filas_por_columna_r1 = [df_recommendation_scores_val_r1[col].nlargest(5).index.tolist() for col in df_recommendation_scores_val_r1.columns]

# Mostrar el resultado
for col_name, top_rows in zip(df_recommendation_scores_val_r1.columns, top_5_filas_por_columna_r1):
    print(f"{col_name}: {top_rows}")

Producto_r1_1: ['Cliente_val_1', 'Cliente_val_3', 'Cliente_val_6', 'Cliente_val_9', 'Cliente_val_4']
Producto_r1_2: ['Cliente_val_3', 'Cliente_val_6', 'Cliente_val_1', 'Cliente_val_4', 'Cliente_val_8']
Producto_r1_3: ['Cliente_val_1', 'Cliente_val_15', 'Cliente_val_9', 'Cliente_val_11', 'Cliente_val_12']
Producto_r1_4: ['Cliente_val_2', 'Cliente_val_1', 'Cliente_val_3', 'Cliente_val_4', 'Cliente_val_6']
Producto_r1_5: ['Cliente_val_3', 'Cliente_val_6', 'Cliente_val_1', 'Cliente_val_9', 'Cliente_val_11']
Producto_r1_6: ['Cliente_val_1', 'Cliente_val_12', 'Cliente_val_9', 'Cliente_val_15', 'Cliente_val_3']
Producto_r1_7: ['Cliente_val_3', 'Cliente_val_6', 'Cliente_val_12', 'Cliente_val_15', 'Cliente_val_4']
Producto_r1_8: ['Cliente_val_3', 'Cliente_val_6', 'Cliente_val_9', 'Cliente_val_12', 'Cliente_val_15']
Producto_r1_9: ['Cliente_val_8', 'Cliente_val_1', 'Cliente_val_3', 'Cliente_val_6', 'Cliente_val_15']
Producto_r1_10: ['Cliente_val_3', 'Cliente_val_6', 'Cliente_val_10', 'Cliente_va